In [1]:
# What are constructors? A constructor is a special method used to initialize an object. __init_ is the constructor method in Python.

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def display_info(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")

# Create an object
book1 = Book("1984", "George Orwell")
book1.display_info()

Title: 1984
Author: George Orwell


In [2]:
# Using instance methods. They operate on instance attribute which is self.

class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.balance = balance

    def deposit(self, amount):
        if amount > 0:
            self.balance += amount
            print(f"Deposited: {amount}")
        else:
            print("Deposit amount must be positive.")

account = BankAccount("Alice", 100)
account.deposit(50)

Deposited: 50


In [3]:
# Class methods vs static methods. Class methods are bound to the class and not the instance of the class. Static methods do not operate on instance or class attributes and do not have access to the instance or class.

class Utility:
    app_version = "1.0"

    @classmethod
    def get_version(cls):
        print(f"App Version : {cls.app_version}")

    @staticmethod
    def greet():
        print(f"Hello, Welcome to the app!")

Utility.get_version()
Utility.greet()

App Version : 1.0
Hello, Welcome to the app!


In [4]:
# Encapsulation
class Account:
    def __init__(self, owner, balance=0):
        self.owner = owner  # Private attribute
        self.__balance = balance  # Private attribute

    def deposit(self, amount):
        if amount > 0:
            self.__balance += amount
            print(f"Deposited: {amount} €, New Balance: {self.__balance} €")
        else:
            print("Deposit amount must be positive.")

    def get_balance(self):
        return self.__balance


account = Account("Bob", 200)
account.deposit(100)
print(f"Account Balance: {account.get_balance()} €")

Deposited: 100 €, New Balance: 300 €
Account Balance: 300 €


In [11]:
# Library Management System

# Step 1: Create a class Library and Book
NUMBER_OF_DAYS = 90
from datetime import datetime, timedelta

class Book:
    def __init__(self, title, author):
        self.title = title
        self.author = author
        self.is_borrowed = False
        self.due_date = ''

    def display_info(self):
        print(f"Title: {self.title}")
        print(f"Author: {self.author}")
        print(f"Status: {'Borrowed' if self.is_borrowed else 'Available'}")
        print(f"Due Date: {self.due_date}")

class Library:
    def __init__(self):
        self.books = []

    # Add a book
    def add_book(self, title, author):
        new_book = Book(title, author.capitalize())
        self.books.append(new_book)
        print(f"Book '{title}' by {author} added to the library.")
        log_transaction("New Book", title, author)
        save_library(title, author)

    # View all books
    def view_books(self):
        if not self.books:
            print("No books in the library.")
        else:
            print("\n--- Library Catalog ---")
            for book in self.books:
                book.display_info()
                print("\n")

    # Borrow a book
    def borrow_book(self, title):
        for book in self.books:
            if book.title == title and not book.is_borrowed:
                book.is_borrowed = True
                due_date = (datetime.now() + timedelta(days=90)).strftime("%Y-%m-%d")
                book.due_date = due_date
                print(f"Book '{title}' has been borrowed. Enjoy Reading.\nYou have up to '{due_date}' to return the book.")
                log_transaction("Borrow", title, author)
                update_library(title, True, due_date)
                return
        print(f"Book '{title}' is not available for borrowing.")

    # Return a book
    def return_book(self, title):
        for book in self.books:
            if book.title == title and book.is_borrowed:
                book.is_borrowed = False
                book.due_date = ''
                print(f"Book '{title}' has been returned. Thanks a lot!")
                log_transaction("Return", title, author)
                update_library(title, False, '')
                return
        print(f"Book '{title}' is not in the library.")

    # Search a book
    def search_book(self, title=None, author=None):
        search_results = []
        print("\n--- Search Result ---")
        if not title and not author:
            print("Either title or author should be filled. Please try again.")
            return
        for book in self.books:
            if (title and book.title == title) or (author and book.author == author):
                search_results.append(book)
        if search_results:
            for result in search_results:
                result.display_info()
                print('\n')
            return
        print(f"No result found for parameters : {'Title :' + title if title else ''}{' - ' if title and author else ''}{'Author :' + author if author else ''}")

# Load from and Save all the saved books in a file
LIBRARY_BOOKS = './assets/library_books.json'
import json

def load_library_books():
    try:
        with open(LIBRARY_BOOKS, 'r') as json_file:
            library_books = json.load(json_file)
        if library_books:
            return library_books
    except FileNotFoundError:
        print("No books found in the library.")
    except Exception as e:
        print(f"An unexpected error occured : {e}.")
    return []

def save_library(title, author):
    try:
        library_books = load_library_books()
        library_books.append(
            {
                "title": title,
                "author": author,
                "status": "Available",
                "due_date": ""
            }
        )
        with open(LIBRARY_BOOKS, 'w') as json_file:
            json.dump(library_books, json_file, indent=2)
    except Exception as e:
        print(f"An unexpected error occured : {e}.")

def update_library(title, is_borrowed, due_date=None):
    try:
        library_books = load_library_books()
        if library_books:
            for book in library_books:
                if book['title'] == title:
                    book['status'] = 'Borrowed' if is_borrowed else 'Available'
                if due_date is not None:
                    book['due_date'] = due_date
        with open(LIBRARY_BOOKS, 'w') as json_file:
            json.dump(library_books, json_file, indent=2)
    except Exception as e:
        print(f"An unexpected error occured : {e}.")

def load_library():
    try:
        with open(LIBRARY_BOOKS, 'r') as json_file:
            library_books = json.load(json_file)
        if library_books:
            library = Library()
            is_borrowed = lambda status: True if status == 'Borrowed' else False
            for book in library_books:
                library_book = Book(book["title"], book["author"])
                library_book.is_borrowed = is_borrowed(book.get('status', 'Available'))
                library_book.due_date = book.get('due_date', '')
                library.books.append(library_book)
            return library
    except FileNotFoundError:
        print("No books found in the library.")
    except Exception as e:
        print(f"An unexpected error occured : {e}.")
    return Library()

# Log Transaction history
from datetime import datetime
LIBRARY_TRANSACTION_HISTORY = './assets/library_transaction_history.json'

def load_history():
    try:
        with open(LIBRARY_TRANSACTION_HISTORY, 'r') as json_file:
            history = json.load(json_file)
        if history:
            return history
    except FileNotFoundError:
        print("No transaction history found for the library.")
    except Exception as e:
        print(f"An unexpected error occured : {e}.")
    return []

def log_transaction(transaction_type, title, author):
    try:
        transaction_history = load_history()
        transaction_history.append(
            {
                "type": transaction_type,
                "title": title,
                "author": author,
                "date": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            }
        )
        with open(LIBRARY_TRANSACTION_HISTORY, 'w') as json_file:
            json.dump(transaction_history, json_file, indent=2)
    except Exception as e:
        print(f"An unexpected error occured : {e}.")

# Main Program

while True:
    library = load_library()

    print("\n------- Library Management System -------")
    print("1. Add Book")
    print("2. Borrow Book")
    print("3. Return Book")
    print("4. View Books")
    print("5. Search a Book")
    print("6. Exit")

    choice = input("Enter your choice (1-6): ").strip()

    if choice == '1':
        title = input("Enter Book title : ").strip().capitalize()
        author = input("Enter Book author : ").strip().capitalize()
        library.add_book(title, author)
    elif choice == '2':
        title = input("Enter Book title: ").strip().capitalize()
        library.borrow_book(title)
    elif choice == '3':
        title = input("Enter Book title: ").strip().capitalize()
        library.return_book(title)
    elif choice == '4':
        library.view_books()
    elif choice == '5':
        while True:
            print("\n------- Search by -------")
            print("1. Title")
            print("2. Author")
            print("3. Both")
            print("4. Exit")

            select = input("Enter your choice (1-4): ").strip()

            if select == '1':
                title = input("Enter Book title: ").strip().capitalize()
                library.search_book(title=title)
            elif select == '2':
                author = input("Enter Book author : ").strip().capitalize()
                library.search_book(author=author)
            elif select == '3':
                title = input("Enter Book title: ").strip().capitalize()
                author = input("Enter Book author : ").strip().capitalize()
                library.search_book(title=title, author=author)
            elif select == '4':
                print("Exiting the Search Menu...")
                break
            else:
                print("Invalid input. Please choose a number between 1 and 4.")
    elif choice == '6':
        print("Exiting the Library Management System. Goodbye!")
        break
    else:
        print("Invalid input. Please choose a number between 1 and 6.")
    



------- Library Management System -------
1. Add Book
2. Borrow Book
3. Return Book
4. View Books
5. Search a Book
6. Exit

------- Search by -------
1. Title
2. Author
3. Both
4. Exit

--- Search Result ---
Title: Ai engineering
Author: Vivian aranha
Status: Available
Due Date: 


Title: Data science mastery
Author: Vivian aranha
Status: Borrowed
Due Date: 



------- Search by -------
1. Title
2. Author
3. Both
4. Exit

--- Search Result ---
No result found for parameters : Title :1

------- Search by -------
1. Title
2. Author
3. Both
4. Exit
Invalid input. Please choose a number between 1 and 4.

------- Search by -------
1. Title
2. Author
3. Both
4. Exit
Exiting the Search Menu...

------- Library Management System -------
1. Add Book
2. Borrow Book
3. Return Book
4. View Books
5. Search a Book
6. Exit

--- Library Catalog ---
Title: Ai engineering
Author: Vivian aranha
Status: Available
Due Date: 


Title: Data science mastery
Author: Vivian aranha
Status: Borrowed
Due Date: 

